# 第3章 大規模言語モデルの基礎

## 3.2 GPT（デコーダ）

### 3.2.4 transformersで使う

In [1]:
!pip install transformers[ja,sentencepiece,torch] pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.0/698.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
  

In [2]:
from transformers import pipeline

In [3]:
%%time
# 後続するテキストを予測するpipelineを作成
generator = pipeline(
    "text-generation", model="abeja/gpt2-large-japanese"
)
# "日本で一番高い山は"に続くテキストを生成
outputs = generator("日本で一番高い山は")
print(outputs[0]["generated_text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/974 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/784k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu


日本で一番高い山は_ 京都・清水寺は紅葉の名所だ « お墓参りに行ったときに気になるのは、お墓がきれいに整えられていたり、墓石の石材が新しいものになっていた
CPU times: user 23.4 s, sys: 13.1 s, total: 36.5 s
Wall time: 56.9 s


### CPU timeのログ
- cpu利用の場合:17.2s, 17.1s
- gpu利用の場合:9.0s, 7.5s

結論：このレベルならまだCPUベースでいいか

## 3.3 BERT・RoBERTa（エンコーダ）

### 3.3.4 transformersで使う

In [4]:
import pandas as pd

# マスクされたトークンを予測するpipelineを作成
fill_mask = pipeline(
    "fill-mask", model="cl-tohoku/bert-base-japanese-v3"
)
masked_text = "日本の首都は[MASK]である"
# [MASK]部分を予測
outputs = fill_mask(masked_text)
# 上位3件をテーブルで表示
display(pd.DataFrame(outputs[:3]))

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

Device set to use cpu


,score,token,token_str,sequence
0,0.884170,12569,東京,日本 の 首都 は 東京 で ある
1,0.024820,12759,大阪,日本 の 首都 は 大阪 で ある
2,0.020864,13017,京都,日本 の 首都 は 京都 で ある


In [7]:
outputs[:10]

[{'score': 0.8841699957847595,
  'token': 12569,
  'token_str': '東京',
  'sequence': '日本 の 首都 は 東京 で ある'},
 {'score': 0.02481999062001705,
  'token': 12759,
  'token_str': '大阪',
  'sequence': '日本 の 首都 は 大阪 で ある'},
 {'score': 0.020863685756921768,
  'token': 13017,
  'token_str': '京都',
  'sequence': '日本 の 首都 は 京都 で ある'},
 {'score': 0.004852684680372477,
  'token': 14499,
  'token_str': '奈良',
  'sequence': '日本 の 首都 は 奈良 で ある'},
 {'score': 0.004590395838022232,
  'token': 14424,
  'token_str': '仙台',
  'sequence': '日本 の 首都 は 仙台 で ある'}]

In [14]:
masked_text = "今日の[MASK]は[MASK]で面白かった。また見に行こう。"
outputs = fill_mask(masked_text, top_k=10)
display(outputs)

[[{'score': 0.08788225054740906,
   'token': 13378,
   'token_str': '舞台',
   'sequence': '[CLS] 今日 の 舞台 は [MASK] で 面白かっ た 。 また 見 に 行こう 。 [SEP]'},
  {'score': 0.0684950053691864,
   'token': 13094,
   'token_str': 'ライブ',
   'sequence': '[CLS] 今日 の ライブ は [MASK] で 面白かっ た 。 また 見 に 行こう 。 [SEP]'},
  {'score': 0.04816514253616333,
   'token': 12632,
   'token_str': '映画',
   'sequence': '[CLS] 今日 の 映画 は [MASK] で 面白かっ た 。 また 見 に 行こう 。 [SEP]'},
  {'score': 0.045846909284591675,
   'token': 13727,
   'token_str': '公演',
   'sequence': '[CLS] 今日 の 公演 は [MASK] で 面白かっ た 。 また 見 に 行こう 。 [SEP]'},
  {'score': 0.03274328634142876,
   'token': 12720,
   'token_str': '試合',
   'sequence': '[CLS] 今日 の 試合 は [MASK] で 面白かっ た 。 また 見 に 行こう 。 [SEP]'},
  {'score': 0.029831787571310997,
   'token': 13651,
   'token_str': 'ステージ',
   'sequence': '[CLS] 今日 の ステージ は [MASK] で 面白かっ た 。 また 見 に 行こう 。 [SEP]'},
  {'score': 0.028001874685287476,
   'token': 14486,
   'token_str': 'ショー',
   'sequence': '[CLS] 今日 の ショー は [MASK] で

複数のmaskを与えると、それぞれ独立に推論するようだ

## 3.4 T5（エンコーダ・デコーダ）

### 3.4.4 transformersで使う

In [ ]:
# text-to-textで生成するpipelineを作成
t2t_generator = pipeline(
    "text2text-generation", model="retrieva-jp/t5-large-long"
)
# マスクされたスパンを予測
masked_text = "江戸幕府を開いたのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

徳川家康


In [ ]:
t2t_generator.tokenizer.convert_tokens_to_ids("<extra_id_1>")

32098

In [ ]:
masked_text = "日本で通貨を発行しているのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

日本銀行


In [ ]:
"日本銀行" in t2t_generator.tokenizer.vocab

False